<a href="https://colab.research.google.com/github/GowsalyaS29/Projects/blob/main/Image_classification_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import os
import zipfile
import random
from shutil import copyfile

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


--2021-08-16 15:02:05--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.53.252.195, 2600:1406:3f:3a0::e59, 2600:1406:3f:384::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.53.252.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M   190MB/s    in 4.3s    

2021-08-16 15:02:10 (184 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [3]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))


12501
12501


In [4]:
try:
    os.mkdir('/tmp/Cats-v-dogs')
    os.mkdir('/tmp/Cats-v-dogs/Training')
    os.mkdir('/tmp/Cats-v-dogs/Testing')
    os.mkdir('/tmp/Cats-v-dogs/Validation')
    os.mkdir('/tmp/Cats-v-dogs/Training/cats')
    os.mkdir('/tmp/Cats-v-dogs/Training/dogs')
    os.mkdir('/tmp/Cats-v-dogs/Testing/cats')
    os.mkdir('/tmp/Cats-v-dogs/Testing/dogs')
except OSError:
    pass

In [5]:
def split(SOURCE, TRAINING, TESTING,SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " No Images")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]
    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

    

CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/Cats-v-dogs/Training/cats/"
TESTING_CATS_DIR = "/tmp/Cats-v-dogs/Testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/Cats-v-dogs/Training/dogs/"
TESTING_DOGS_DIR = "/tmp/Cats-v-dogs/Testing/dogs/"
split_size = .9
split(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)


666.jpg No Images
11702.jpg No Images


In [6]:
print(len(os.listdir('/tmp/Cats-v-dogs/Training/cats/')))
print(len(os.listdir('/tmp/Cats-v-dogs/Training/dogs/')))
print(len(os.listdir('/tmp/Cats-v-dogs/Testing/cats/')))
print(len(os.listdir('/tmp/Cats-v-dogs/Testing/dogs/')))


11250
11250
1250
1250


In [95]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [97]:
TRAINING_DIR = "/tmp/Cats-v-dogs/Training/"
image_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator= image_train.flow_from_directory(batch_size=250,
                                                     directory=TRAINING_DIR,
                                                     shuffle=True,
                                                     target_size=(100,100),
                                                     class_mode='binary')

Found 22498 images belonging to 2 classes.


In [98]:
VALIDATION_DIR = "/tmp/Cats-v-dogs/Testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=250,
                                                              class_mode='binary',
                                                              target_size=(100, 100))


Found 2500 images belonging to 2 classes.


In [99]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout
from tensorflow.keras.models import Model

In [100]:
base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(100,100,3))


94773248/94765736 [==============================] - 1s 0us/step


In [101]:
x = base_model.output

In [102]:
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
base_model.layers[0].trainable = True


In [103]:
model = Model(inputs=base_model.input, outputs=predictions)


In [104]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=90,
                    validation_data=validation_generator, validation_steps=6)

In [92]:
model.save('model2.h5')

In [93]:
os.stat('model2.h5').st_size


26466472

In [ ]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()


In [94]:
import time
start_main = time.time()

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  #print(images)
  img = image.load_img(path, target_size=(100, 100))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  
  start = time.time()

  classes = model.predict(images)
  end = time.time()
  print("prediction time : " + str(end-start))

  print(classes[0])
  if prediction[0]<0.5:
    pred_name='Cat'
  else:
    pred_name='Dog'
  print(pred_name)
  end_main = time.time()
  print("Overall Time : " + str(end_main-start_main))

 

Saving 6.jpg to 6 (3).jpg
prediction time : 0.10808300971984863
[0.]
Cat
Overall Time : 10.404144287109375


In [39]:
!pip install opencv-python

In [12]:
!pip install ColabCode

In [13]:
!pip install fastapi

     |████████████████████████████████| 52 kB 866 kB/s 
     |████████████████████████████████| 10.1 MB 10.5 MB/s 
     |████████████████████████████████| 60 kB 8.9 MB/s 


In [19]:
from colabcode import ColabCode
from fastapi import FastAPI

In [20]:
cc = ColabCode(port=8000, code=False)

In [21]:
app = FastAPI()

@app.get("/")
async def read_root():
  return {"message": "Dod vs Cat classification"}

In [27]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://3b9c3725f4eb.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [64]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     67.167.145.64:0 - "GET / HTTP/1.1" 200 OK
INFO:     67.167.145.64:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [64]


In [14]:
!pip install python-multipart

  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=5765acdea37603fcb0c9eebc4c8ac5180a87e59709f06ab94bed3787378132ef
  Stored in directory: /root/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built python-multipart


In [22]:
app = FastAPI()

from tensorflow.keras.models import load_model
@app.on_event("startup")
def load_model():
    global model
    model = tf.keras.models.load_model('model.h5')



In [23]:
input_shape = model.layers[0].input_shape


In [24]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from PIL import Image
from keras.preprocessing import image
from pydantic import BaseModel
from typing import List
import io
import numpy as np
import sys


In [25]:
class Prediction(BaseModel):
  filename: str
  contenttype: str
  prediction: float


In [26]:
@app.post('/prediction/', response_model=Prediction)
async def prediction_route(file: UploadFile = File(...)):

  # Ensure that this is an image
  if file.content_type.startswith('image/') is False:
    raise HTTPException(status_code=400, detail=f'File \'{file.filename}\' is not an image.')

  try:
    # Read image contents
    contents = await file.read()
    pil_image = Image.open(io.BytesIO(contents))

    # Resize image to expected input shape
    pil_image = pil_image.resize((100, 100))
    if pil_image.mode == 'RGBA':
      pil_image = pil_image.convert('RGB')

    x = image.img_to_array(pil_image)
    x = np.expand_dims(x, axis=0)
  #print(x)

    images = np.vstack([x])
 
    # Convert from RGBA to RGB *to avoid alpha channels*
   
    
    # Convert image into numpy format
    
    # Generate prediction
    predictions = model.predict(images)
    prediction = predictions[0]

    return {
      'filename': file.filename,
      'contenttype': file.content_type,
      'prediction': prediction
    }
  except:
    e = sys.exc_info()[1]
    raise HTTPException(status_code=500, detail=str(e))
    

In [28]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://ce036108ad22.ngrok.io" -> "http://localhost:8000"


INFO:     Started server process [64]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     67.167.145.64:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     67.167.145.64:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     206.180.159.37:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     67.167.145.64:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     67.167.145.64:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     63.88.73.70:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     63.88.73.70:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     67.167.145.64:0 - "POST /prediction/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [64]
